# ML final project

## Download dataset and libraries

In [ ]:
!pip install scikit-learn
!pip install transformers
!pip install xgboost
!pip install torch
!pip install numpy

!wget -O dataset.zip "https://www.kaggle.com/api/v1/datasets/download/sbhatti/financial-sentiment-analysis"
!unzip -p dataset.zip > dataset.csv

## Load dataset

In [ ]:
import pandas as pd

df = pd.read_csv('dataset.csv')
df.head()

## Prepare dataset


### Download already created embeddings
If you want to create the embeddings yourself, skip this cell and continue with the next cells. They will produce embeddings using the BERT-large-uncased model on a GPU.

In [4]:
# !wget https://github.com/misosvec/ml-course-project/raw/refs/heads/main/cls_embeddings.npy
# !wget https://github.com/misosvec/ml-course-project/raw/refs/heads/main/sum_embeddings.npy
# !wget https://github.com/misosvec/ml-course-project/raw/refs/heads/main/mean_embeddings.npy
# !wget https://github.com/misosvec/ml-course-project/raw/refs/heads/main/max_embeddings.npy

import numpy as np

sum_embeddings = np.load('sum_embeddings.npy')
max_embeddings = np.load('max_embeddings.npy')
cls_embeddings = np.load('cls_embeddings.npy')
mean_embeddings = np.load('mean_embeddings.npy')



### Create embeddings

In [3]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-large-cased")
model = AutoModel.from_pretrained("google-bert/bert-large-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

#### Tokenize X

In [6]:
X = tokenizer(
    df["Sentence"].tolist(),
    padding=True,
    return_tensors="pt"
)

#### Generte X embeddings using the BERT on GPU

In [7]:
from torch.utils.data import DataLoader, TensorDataset
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Using device: {device}")

all_sentences_embeddings = []

# dataset will be moved to GPU
input_ids = X['input_ids'].to(device)
attention_mask = X['attention_mask'].to(device)
dataset = TensorDataset(input_ids, attention_mask)

# we are using batches due to memory capacity
batch_size = 1024
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

for batch in dataloader:
  input_ids_batch, attention_mask_batch = batch
  input_ids_batch = input_ids_batch.to(device)
  attention_mask_batch = attention_mask_batch.to(device)
  # using no_grad for another memory optimization
  with torch.no_grad():
    outputs = model(input_ids=input_ids_batch, attention_mask=attention_mask_batch)
    all_sentences_embeddings.append(outputs.last_hidden_state)

# Combine all batches
all_sentences_embeddings = torch.cat(all_sentences_embeddings, dim=0)

Using device: cuda


In [13]:
import numpy as np

sum_embeddings = all_sentences_embeddings.sum(dim=1)
max_embeddings, _ = all_sentences_embeddings.max(dim=1)
cls_embeddings = all_sentences_embeddings[:, 0, :]
mean_embeddings = all_sentences_embeddings.mean(dim=1)

# bring it back to cpu
sum_embeddings = sum_embeddings.cpu().numpy()
max_embeddings = max_embeddings.cpu().numpy()
cls_embeddings = cls_embeddings.cpu().numpy()
mean_embeddings = mean_embeddings.cpu().numpy()

np.save("cls_embeddings.npy", cls_embeddings)
np.save("sum_embeddings.npy", sum_embeddings)
np.save("max_embeddings.npy", max_embeddings)
np.save("mean_embeddings.npy", mean_embeddings)

### Encode target variable y

In [13]:
X = cls_embeddings
# X = sum_embeddings
# X = max_embeddings
# X = mean_embeddings

y = df["Sentiment"].to_numpy()
y[y == 'positive'] = 0
y[y == 'negative'] = 1
y[y == 'neutral'] = 2
y = y.astype(int)

### Datset train, valid, test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=42)

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

## Evaluate function

In [16]:
def evaluate_models(models, X_test, y_test):
  for model in models:
    print(model.score(X_test, y_test))

def train_valid_eval(model, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val):
  print(f"{type(model).__name__} training acccuracy: {accuracy_score(y_train, clf.predict(X_train))}")
  print(f"{type(model).__name__} validation accuracy: {accuracy_score(y_val, clf.predict(X_val))}")

## Baseline model

In [ ]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
evaluate_models([dummy_clf], X_test, y_test)

0.5286569717707442


In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(

)
clf = clf.fit(X_train, y_train)
evaluate_models([clf], X_test, y_test)

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [17]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
evaluate_models([rfc], X_test, y_test)

0.6458511548331908


In [18]:
from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=50, learning_rate=1.0, max_depth=1, random_state=0)
gbc.fit(X_train, y_train)
evaluate_models([gbc], X_test, y_test)

0.7031650983746792


In [22]:
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report


clf = xgb.XGBClassifier(
    n_estimators=5,
    max_depth=3,
    num_class=3,               # Number of classes
    learning_rate= 1,
)
clf.fit(X_train, y_train)
evaluate_models([clf], X_test, y_test)

0.6672369546621043


In [23]:
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier(estimator=SVC(), n_estimators=40)
bc.fit(X_train, y_train)
evaluate_models([bc], X_test, y_test)

0.7288280581693756


In [18]:
import torch
import numpy as np

# Step 1: Create a PyTorch tensor
tensor = torch.randn(3, 3)

numpy_array = tensor.numpy()

np.save('tensor_data.npy', numpy_array)


loaded_numpy_array = np.load('td.npy')

print(loaded_numpy_array)


[[ 0.62769145 -0.4939803  -0.5412137 ]
 [-0.10107227 -0.07990402 -1.5660136 ]
 [-1.2541329  -0.3354928  -2.046507  ]]
